In [1]:
#pip install gazpacho

In [2]:
from gazpacho import get, Soup
import numpy
import pandas
import requests, bs4
import re, os
import time

In [3]:
# Verifying that the page request is valid
from gazpacho import get, Soup

url = 'https://omegayeast.com/yeast/ales/east-coast-ale'

# fetch step
html = get(url)

# extract step
soup = Soup(html)

### Pull The Strain Name

In [4]:
# Pull The Strain Name

def pull_strain_name(soup):
    strain_name = soup.find('h1', {'class':'heading-primary__heading h-transition-fade h-transition-delay-2'})
    x = str(strain_name)
    print(x)
    start = x.find('delay-2')
    end = x.find('</h1>')
    strain = x[start+10:end]
    # print(strain)
    return strain

In [5]:
pull_strain_name(soup)

<h1 class="heading-primary__heading h-transition-fade h-transition-delay-2">  East Coast Ale 
</h1>


' East Coast Ale \n'

### Pull The Strain Number

In [6]:
# use find() to parse a specific HTML
gaz_data = soup.find('div', {'class':'l-product-intro__content2'})
gaz_data

<div class="l-product-intro__content2">
                    
  
  <div class="heading-primary  ">

              <p class="heading-primary__subhead h-transition-fade h-transition-delay-1">
        OYL-032
      </p>
    
        <h1 class="heading-primary__heading h-transition-fade h-transition-delay-2">  East Coast Ale 
</h1>

        
  </div>

        

  <div class="rich-text h-transition-fade h-transition-delay-1">
        <p>Thought to be the famous Bostonian strain. It is suitable for a broad range of beer styles with an American spin. Using <a href="https://omegayeast.com/yeast/ales/west-coast-ale-i">West Coast Ale I (OYL-004)</a> for comparison, it is a little tart and slightly muting of hop bitterness. Slightly less flocculent and attenuative, too, but every bit as versatile, carrying an overall clean and neutral flavor character.</p>


  </div>

        

  <div class="rich-text rich-text--small h-transition-fade h-transition-delay-1">
        <p>Omega Yeast’s metabolically 

In [7]:
# Pull The Strain Number

def pull_strain_number(soup):
    strain_number = soup.find('div', {'class':'l-product-intro__content2'})
    x = str(strain_number)
    start = x.find('OYL')
    number = x[start:start+7]
    # print(number)
    return number

In [8]:
pull_strain_number(soup)

'OYL-032'

### Pull The Description

In [9]:
# Pull and save the description

def pull_strain_description(soup):
    description = soup.find('div', {'class':'rich-text h-transition-fade h-transition-delay-1'})
    #print(description)
    x= str(description)
    start = x.find('<p>')
    start = start + 3
    end = x.find('</p>')
    description = x[start:end]
    # print(description)
    return description

In [10]:
pull_strain_description(soup)

'\n    Thought to be the famous Bostonian strain. It is suitable for a broad range of beer styles with an American spin. Using \n    <a href="https://omegayeast.com/yeast/ales/west-coast-ale-i">West Coast Ale I (OYL-004)</a>\n     for comparison, it is a little tart and slightly muting of hop bitterness. Slightly less flocculent and attenuative, too, but every bit as versatile, carrying an overall clean and neutral flavor character.\n  '

### Get Stats on Strain

In [11]:
def pull_stats(soup):
    raw = soup.find('div', {'class':'product-details h-transition-fade product-details--2col'})
    x= str(raw)
    # print(x)
    fs = x.find('>Flocculation</p>') + 57
    fe = x.find('>Flocculation</p>') + 65
    flocculation = x[fs:fe]
    # print(flocculation)

    attens = x.find('>Attenuation</p>') + 55
    attene = x.find('>Attenuation</p>') + 65
    attenuation = x[attens:attene]
    # print(attenuation)

    abve = x.find('>Alcohol Tolerance</p>') + 60
    abvs = x.find('>Alcohol Tolerance</p>') + 65
    abv = x[abve:abvs]
    # print(abv)

    temps = x.find('>Temperature Range</p>') + 60
    tempe = x.find('>Temperature Range</p>') + 82
    temp = x[temps:tempe]
    # print(temp)

    stas = x.find('>Diastatic</p>') + 53
    stae = x.find('>Diastatic</p>') + 59
    STA1 = x[stas:stae]
    # print(temp)
    
    
    return(flocculation, attenuation, abv, temp, STA1)

In [12]:
pull_stats(soup)

('Medium-L', '>70-75%</d', '">10%', '">68–73° F (20–23°\xa0C)<', '>No</d')

### Pull all the urls

In [13]:
import pandas as pd
urls = pd.read_csv('data/omega_all.csv')
urls

,Address,Content Type,Status Code,Status,Indexability,Indexability Status,Title 1,Title 1 Length,Title 1 Pixel Width,Meta Description 1,...,Grammar Errors,Hash,Response Time,Last Modified,Redirect URL,Redirect Type,Cookies,HTTP Version,URL Encoded Address,Crawl Timestamp
0,https://omegayeast.com/yeast/lagers/bayern-lager,text/html,200,OK,Indexable,NaN,Bayern Lager | Omega Yeast,26,250,"This clean, crisp, lager strain of Bavarian or...",...,NaN,98c34d6d7a2ec28e70b5f35ada0f98cf,0.049,"Tue, 22 Aug 2023 00:17:12 GMT",NaN,NaN,NaN,1.1,https://omegayeast.com/yeast/lagers/bayern-lager,9/14/2023 12:26
1,https://omegayeast.com/resources/probrewing,text/html,200,OK,Indexable,NaN,ProbrewingResources | Omega Yeast,33,327,Let's talk nerdy.,...,NaN,2d27156ec327f792d03955385f4f367c,0.044,"Mon, 21 Aug 2023 21:34:50 GMT",NaN,NaN,NaN,1.1,https://omegayeast.com/resources/probrewing,9/14/2023 12:26
2,https://omegayeast.com/yeast/lagers/lunar-crush,text/html,200,OK,Indexable,NaN,Lunar Crush | Omega Yeast,25,241,"Put a spin on lager brewing with Lunar Crush®,...",...,NaN,80274e4e72633e8e36c449539778187e,0.147,"Wed, 23 Aug 2023 20:12:23 GMT",NaN,NaN,NaN,1.1,https://omegayeast.com/yeast/lagers/lunar-crush,9/14/2023 12:26
3,https://omegayeast.com/resources/retailer,text/html,200,OK,Indexable,NaN,RetailerResources | Omega Yeast,31,293,Let's talk nerdy.,...,NaN,2f674c2e816a713d9ed61303f66b4813,0.156,"Tue, 22 Aug 2023 00:16:58 GMT",NaN,NaN,NaN,1.1,https://omegayeast.com/resources/retailer,9/14/2023 12:26
4,https://omegayeast.com/careers,text/html,200,OK,Indexable,NaN,Careers | Omega Yeast,21,202,Omega Yeast is made up of a handful of microbi...,...,NaN,ee148dd953ddda91e56c5dcc30ebc6c4,0.042,"Tue, 22 Aug 2023 17:17:05 GMT",NaN,NaN,NaN,1.1,https://omegayeast.com/careers,9/14/2023 12:26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,https://omegayeast.com/yeast/ales/british-ale-ii,text/html,200,OK,Indexable,NaN,British Ale II | Omega Yeast,28,239,"Like British Ale I (OYL-006) , this English br...",...,NaN,f1aa59500526bd7043acdd9a30b8c79c,0.033,"Wed, 23 Aug 2023 23:09:35 GMT",NaN,NaN,NaN,1.1,https://omegayeast.com/yeast/ales/british-ale-ii,9/14/2023 12:26
197,https://omegayeast.com/yeast/ales/british-ale-iv,text/html,200,OK,Indexable,NaN,British Ale IV | Omega Yeast,28,246,"Nuances of apple, clover honey and pear and a ...",...,NaN,c62a19b343a761e977b7a5ab033b920c,0.030,"Wed, 23 Aug 2023 18:31:07 GMT",NaN,NaN,NaN,1.1,https://omegayeast.com/yeast/ales/british-ale-iv,9/14/2023 12:26
198,https://omegayeast.com/yeast/lagers/american-p...,text/html,200,OK,Indexable,NaN,American Pilsner | Omega Yeast,30,284,Thought to be from the most popular beer in Am...,...,NaN,d6413ff9afd51a63cb653afc2cfa6cbf,0.031,"Tue, 22 Aug 2023 14:21:48 GMT",NaN,NaN,NaN,1.1,https://omegayeast.com/yeast/lagers/american-p...,9/14/2023 12:26
199,https://omegayeast.com/yeast/belgian-ales/belg...,text/html,200,OK,Indexable,NaN,Belgian Ale DK | Omega Yeast,28,263,A mild Belgian-charactered ale yeast with ligh...,...,NaN,aa2ab6375e42d17fb2a9cd2abbe231e4,0.037,"Wed, 23 Aug 2023 17:13:32 GMT",NaN,NaN,NaN,1.1,https://omegayeast.com/yeast/belgian-ales/belg...,9/14/2023 12:26


In [14]:
url = urls.Address.values.tolist()
url

['https://omegayeast.com/yeast/lagers/bayern-lager',
 'https://omegayeast.com/resources/probrewing',
 'https://omegayeast.com/yeast/lagers/lunar-crush',
 'https://omegayeast.com/resources/retailer',
 'https://omegayeast.com/careers',
 'https://omegayeast.com/yeast/ales/sundew',
 'https://omegayeast.com/news',
 'https://omegayeast.com/yeast/bretts-blends-bacteria/brett-blend-2-bit-o-funk',
 'https://omegayeast.com/propper/propper-seltzer',
 'https://omegayeast.com/yeast/ales/kolsch-ii-dko',
 'https://omegayeast.com/yeast/bretts-blends-bacteria/brett-blend-3-bring-on-da-funk',
 'https://omegayeast.com/propper/propper-starter',
 'https://omegayeast.com/yeast/lagers/bayern-lager-dko',
 'https://omegayeast.com/yeast/belgian-ales/saisonstein',
 'https://omegayeast.com/yeast/ales/point-loma-dko',
 'https://omegayeast.com/yeast/bretts-blends-bacteria/c2c-american-farmhouse',
 'https://omegayeast.com/yeast/bretts-blends-bacteria/all-the-bretts',
 'https://omegayeast.com/yeast/ales/bananza',
 'h

In [15]:
import pandas as pd

def loadTable(soup):
    strain = pull_strain_name(soup)
    #pullMetrics(soup)
    number = pull_strain_number(soup)
    flocculation, attenuation, abv, temp, STA1 = pull_stats(soup)
    description = pull_strain_description(soup)
    entry = {'Strain': [strain],
             'Number' : [number],
        'Attenuation' : [attenuation],
        'Flocculation' : [flocculation],
        'Alcohol_Tolerance' : [abv],
        'Fermentation_Temperature' : [temp],
        'STA1' : [STA1],
        'Description' : [description]}
    df = pd.DataFrame(entry)
    return df

In [16]:
loadTable(soup)

<h1 class="heading-primary__heading h-transition-fade h-transition-delay-2">  East Coast Ale 
</h1>


,Strain,Number,Attenuation,Flocculation,Alcohol_Tolerance,Fermentation_Temperature,STA1,Description
0,East Coast Ale \n,OYL-032,>70-75%</d,Medium-L,""">10%",""">68–73° F (20–23° C)<",>No</d,\n Thought to be the famous Bostonian strai...


In [17]:
df = loadTable(soup)
df1 = df
df1

<h1 class="heading-primary__heading h-transition-fade h-transition-delay-2">  East Coast Ale 
</h1>


,Strain,Number,Attenuation,Flocculation,Alcohol_Tolerance,Fermentation_Temperature,STA1,Description
0,East Coast Ale \n,OYL-032,>70-75%</d,Medium-L,""">10%",""">68–73° F (20–23° C)<",>No</d,\n Thought to be the famous Bostonian strai...


In [18]:
for i in url:
  print(i)
  # fetch step
  html = get(i)

  # extract step
  soup = Soup(html)
  df = loadTable(soup)
  df1 = pd.concat([df1, df])

https://omegayeast.com/yeast/lagers/bayern-lager
<h1 class="heading-primary__heading h-transition-fade h-transition-delay-2">  Bayern Lager 
</h1>
https://omegayeast.com/resources/probrewing
None
https://omegayeast.com/yeast/lagers/lunar-crush
<h1 class="heading-primary__heading h-transition-fade h-transition-delay-2">  Lunar Crush 
</h1>
https://omegayeast.com/resources/retailer
None
https://omegayeast.com/careers
None
https://omegayeast.com/yeast/ales/sundew
<h1 class="heading-primary__heading h-transition-fade h-transition-delay-2">  Sundew 
</h1>
https://omegayeast.com/news
None
https://omegayeast.com/yeast/bretts-blends-bacteria/brett-blend-2-bit-o-funk
<h1 class="heading-primary__heading h-transition-fade h-transition-delay-2">  Brett Blend #2 Bit O’ Funk 
</h1>
https://omegayeast.com/propper/propper-seltzer
<h1 class="heading-primary__heading h-transition-fade h-transition-delay-2">  Propper Seltzer
</h1>
https://omegayeast.com/yeast/ales/kolsch-ii-dko
<h1 class="heading-primary

None
https://omegayeast.com/news/introducing-lunar-crush
None
https://omegayeast.com/yeast/bretts-blends-bacteria/brettanomyces-bruxellensis
<h1 class="heading-primary__heading h-transition-fade h-transition-delay-2">  Brett. Bruxellensis 
</h1>
https://omegayeast.com/dried-yeast-calculator
None
https://omegayeast.com/yeast/lagers/pilsner-i
<h1 class="heading-primary__heading h-transition-fade h-transition-delay-2">  Pilsner I 
</h1>
https://omegayeast.com/news/six-new-homebrew-strains
None
https://omegayeast.com/all-about-our-thiolized-yeast-series
None
https://omegayeast.com/yeast/ales/british-ale-vi
<h1 class="heading-primary__heading h-transition-fade h-transition-delay-2">  British Ale VI 
</h1>
https://omegayeast.com/yeast/ales/west-coast-ale-i
<h1 class="heading-primary__heading h-transition-fade h-transition-delay-2">  West Coast Ale I 
</h1>
https://omegayeast.com/news/hands-on-review-propper-starter-canned-starter-wort
None
https://omegayeast.com/news/beer-made-from-yeast-blo

None
https://omegayeast.com/news/concentrated-canned-wort-makes-brewing-easier-for-homebrewers
None
https://omegayeast.com/yeast/bretts-blends-bacteria/brettanomyces-claussenii
<h1 class="heading-primary__heading h-transition-fade h-transition-delay-2">  Brett. Claussenii 
</h1>
https://omegayeast.com/news/new-year-nine-new-homebrew-strains
None
https://omegayeast.com/yeast/ales/american-wheat
<h1 class="heading-primary__heading h-transition-fade h-transition-delay-2">  American Wheat 
</h1>
https://omegayeast.com/yeast/belgian-ales/belgian-ale-d
<h1 class="heading-primary__heading h-transition-fade h-transition-delay-2">  Belgian Ale D 
</h1>
https://omegayeast.com/yeast/lagers/german-lager-ii
<h1 class="heading-primary__heading h-transition-fade h-transition-delay-2">  German Lager II 
</h1>
https://omegayeast.com/yeast/belgian-ales/belgian-ale-r
<h1 class="heading-primary__heading h-transition-fade h-transition-delay-2">  Belgian Ale R 
</h1>
https://omegayeast.com/yeast/belgian-ale

In [19]:
df1

,Strain,Number,Attenuation,Flocculation,Alcohol_Tolerance,Fermentation_Temperature,STA1,Description
0,East Coast Ale \n,OYL-032,>70-75%</d,Medium-L,""">10%",""">68–73° F (20–23° C)<",>No</d,\n Thought to be the famous Bostonian strai...
0,Bayern Lager \n,OYL-114,>72-76%</d,Medium</,""">9%",""">51–62° F (11–17° C)<",>No</d,"This clean, crisp, lager strain of Bavarian or..."
0,,e,,,,,,n
0,Lunar Crush \n,OYL-403,>70-78%</d,Medium</,""">9%",""">50–65° F (10–18° C)<",>No</d,Put a spin on lager brewing with Lunar Crush<s...
0,,e,,,,,,n
...,...,...,...,...,...,...,...,...
0,British Ale II \n,OYL-007,>68-72%</d,Medium-H,""">10%",""">64–75° F (18–24° C)<",>No</d,"\n Like \n <a href=""https://omegayeast.c..."
0,British Ale IV \n,OYL-010,>69-76%</d,Medium-L,""">10%",""">69–76° F (21–24° C)<",>No</d,"Nuances of apple, clover honey and pear and a ..."
0,American Pilsner \n,OYL-102,>71-75%</d,Medium</,""">9%",""">48–56° F (9–13° C)</",>No</d,Thought to be from the most popular beer in Am...
0,Belgian Ale DK \n,OYL-049,>73-80%</d,Medium</,""">10%",""">67–70° F (19–21° C)<",>No</d,A mild Belgian-charactered ale yeast with ligh...


# Pulling all the urls that are needed

In [20]:
df1.to_csv('Omega_Strains2024.csv')